# HW_05 : CNN Нейросети в обработке текста - заглушка

Задание из 2-х частей.
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации - выбить auc выше 0.95
2. Предобучаем word2vec и его эмбединга инициализируем сетку, как влияет на качество?


In [145]:
import warnings
warnings.filterwarnings("ignore")

In [146]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
tqdm.pandas()

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

DATA_PATH = '../data/отзывы за лето.xls'

In [147]:
# глобальные переменнные для предобработки
max_words = 2000
max_len = 40
num_classes = 1

In [148]:
df = pd.read_excel(DATA_PATH, parse_dates=[2])
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


### Бинарная классификация

In [149]:
df_bin = df[df['Rating']!=3]

In [150]:
df_bin['Target'] = (df['Rating']>3).astype('uint8')

In [151]:
# распределение не очень, но рабочее
df_bin.Target.value_counts(normalize=True)

1    0.846871
0    0.153129
Name: Target, dtype: float64

#### Препроцессинг

In [152]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [153]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_bin['Content'] = df_bin['Content'].apply(preprocess_text)

In [154]:
# разобьем датасет на трайн-валид-тест 70-15-15
train, test = train_test_split(df_bin, test_size=0.7, shuffle=True, stratify = df_bin['Target'], random_state=42)
valid, test = train_test_split(test, test_size=0.5, shuffle=True, random_state=42)

In [155]:
train_corpus = " ".join(train["Content"])
train_corpus = train_corpus.lower()

In [156]:
tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()]

In [157]:
# наиболее часто встречающиеся токены
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [158]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [159]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [160]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in test["Content"]], dtype=np.int32)
x_valid = np.asarray([text_to_sequence(text, max_len) for text in valid["Content"]], dtype=np.int32)

In [161]:
x_train.shape

(5924, 40)

In [162]:
x_train[5]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0, 651,  18, 122, 190, 964, 964,
       964], dtype=int32)

### Модель

In [163]:
import tensorflow as tf
import tensorflow.keras as k
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import EarlyStopping 

In [164]:
num_classes = 2
y_train = k.utils.to_categorical(train["Target"], num_classes)
y_test = k.utils.to_categorical(test["Target"], num_classes)
y_valid = k.utils.to_categorical(valid["Target"], num_classes)

In [165]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [166]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [167]:
epochs = 50
batch_size = 512
# print_batch_n = 100

In [168]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss', patience=3)  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/50
11/11 [==============================] - 1s 107ms/step - loss: 0.6700 - accuracy: 0.6753 - val_loss: 0.6046 - val_accuracy: 0.8668
Epoch 2/50
11/11 [==============================] - 1s 102ms/step - loss: 0.5513 - accuracy: 0.8462 - val_loss: 0.4560 - val_accuracy: 0.8668
Epoch 3/50
11/11 [==============================] - 1s 94ms/step - loss: 0.4164 - accuracy: 0.8467 - val_loss: 0.3532 - val_accuracy: 0.8668
Epoch 4/50
11/11 [==============================] - 1s 95ms/step - loss: 0.3508 - accuracy: 0.8467 - val_loss: 0.3002 - val_accuracy: 0.8702
Epoch 5/50
11/11 [==============================] - 1s 102ms/step - loss: 0.2679 - accuracy: 0.8899 - val_loss: 0.2419 - val_accuracy: 0.9039
Epoch 6/50
11/11 [==============================] - 1s 96ms/step - loss: 0.1988 - accuracy: 0.9285 - val_loss: 0.2198 - val_accuracy: 0.9140
Epoch 7/50
11/11 [==============================] - 1s 96ms/step - loss: 0.1621 - accuracy: 0.9435 - val_loss: 0.2110 - val_accuracy: 0.9157
Epoch 8/50

In [169]:
score = model.evaluate(x_valid, y_valid, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 0s 26ms/step - loss: 0.2275 - accuracy: 0.9116


Test score: 0.22754661738872528
Test accuracy: 0.9116030335426331


In [170]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 0s 25ms/step


In [171]:
print(f'Test score: {round(roc_auc_score(y_test, results),3)}')

Test score: 0.951
